In [1]:
import pickle
# load a file to a variable
with open('C:/ntpu_project/dicom_128x128.pickle', 'rb') as file:
    dicom_list = pickle.load(file)

print(f'dicom length = {len(dicom_list)}')

dicom length = 118209


In [2]:
# model training
import traceback
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical

In [8]:
# prepair data for CNN
x = list()
y = list()
all_imgs = list()

while dicom_list:
    dicom = dicom_list.pop()
    all_imgs.append(np.expand_dims(dicom.standardlize(), axis=2))
    temp_y = []
    if dicom.has_nodule:
        temp_y.append(1)
    else:
        temp_y.append(0)
    y.append(temp_y)
    
x = np.stack(all_imgs)
# all_imgs = [np.expand_dims(dicom.standardlize(), axis=2) for dicom in dicom_list]

In [9]:
# release memory
del all_imgs
del dicom_list

In [10]:
print(np.shape(x))
print(np.shape(y))
print(np.sum(y))

(118209, 128, 128, 1)
(118209, 1)
3194


In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2100, shuffle=True)
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)

In [12]:
# release memory
del x
del y

In [26]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(y_train_onehot))
print(np.shape(x_test))
print(np.shape(y_test))
print(np.shape(y_test_onehot))

(94567, 128, 128, 1)
(94567, 1)
(94567, 2)
(23642, 128, 128, 1)
(23642, 1)
(23642, 2)


In [24]:
non_nodule_count = np.sum(y_test)
print(f'non_nodule_count = {non_nodule_count}')
xx_test = list()
yy_test = list()
nodule_count = 0
for i, y in enumerate(y_test):
    if y[0] == 1:
        yy_test.append(y)
        xx_test.append(x_test[i])
    elif y[0] == 0:
        if nodule_count < non_nodule_count:
            yy_test.append(y)
            xx_test.append(x_test[i])
            nodule_count += 1
            
print(f'np.sum(yy_test) = {np.sum(yy_test)}')
print(np.shape(xx_test))
print(np.shape(yy_test))
yy_test_onehot = to_categorical(yy_test)
print(np.shape(yy_test_onehot))

non_nodule_count = 659
np.sum(yy_test) = 659
(1318, 128, 128, 1)
(1318, 1)
(1318, 2)


In [14]:
import tensorflow.compat.v1 as tf
# import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75, allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
tf.keras.backend.set_session(sess)

C:\Users\jason\Anaconda3\envs\tensorflow-gpu-v2.1.0\lib\site-packages\tensorflow_core\python\client\session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [15]:
input_shape = np.shape(x_train[0])
print(input_shape)
batch_size = 10
epochs = 10
num_classes = 2

(128, 128, 1)


In [16]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=16,即 output space 的深度, Kernal Size: 5x5, activation function 採用 relu
model.add(Conv2D(filters=16, 
                 kernel_size=(2, 2),
                 padding="same",
                 activation='relu',
                 input_shape=input_shape))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, 
                 kernel_size=(2, 2),
                 padding="same",
                 activation='relu',
                 input_shape=input_shape))

model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# Flatten層把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add(Flatten())

# 全連接層: 128個output
model.add(Dense(128, activation='relu'))

# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# 使用 softmax activation function，將結果分類
model.add(Dense(num_classes, activation='softmax'))

# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

print(model.summary()) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [17]:
# 進行訓練, 訓練過程會存在 train_history 變數中

train_history = model.fit(x_train, 
                          y_train_onehot,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1)

# 顯示損失函數、訓練成果(分數)
score = model.evaluate(x_test, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 94567 samples
Epoch 1/10
94567/94567 [==============================] - 45s 476us/sample - loss: 0.1273 - accuracy: 0.9732
Epoch 2/10
94567/94567 [==============================] - 42s 442us/sample - loss: 0.1179 - accuracy: 0.9732
Epoch 3/10
94567/94567 [==============================] - 42s 446us/sample - loss: 0.1135 - accuracy: 0.9732
Epoch 4/10
94567/94567 [==============================] - 42s 439us/sample - loss: 0.1096 - accuracy: 0.9732
Epoch 5/10
94567/94567 [==============================] - 42s 440us/sample - loss: 0.1047 - accuracy: 0.9732
Epoch 6/10
94567/94567 [==============================] - 42s 440us/sample - loss: 0.0994 - accuracy: 0.9737
Epoch 7/10
94567/94567 [==============================] - 42s 440us/sample - loss: 0.0938 - accuracy: 0.9743
Epoch 8/10
94567/94567 [==============================] - 42s 440us/sample - loss: 0.0875 - accuracy: 0.9757
Epoch 9/10
94567/94567 [==============================] - 42s 443us/sample - loss: 0.0850 - accuracy: 0.9

In [27]:
score = model.evaluate(x_test, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07556435934050254
Test accuracy: 0.9766517


In [41]:
print(np.shape(x_test))
print(np.shape(y_test_onehot))
print(np.shape(x_test[0]))
print(np.shape(y_test_onehot[0]))

(23642, 128, 128, 1)
(23642, 2)
(128, 128, 1)
(2,)


In [40]:
print(np.shape(xx_test))
print(np.shape(yy_test_onehot))
print(np.shape(xx_test[0]))
print(np.shape(yy_test_onehot[0]))

(1318, 128, 128, 1)
(1318, 2)
(128, 128, 1)
(2,)


In [44]:

score = model.evaluate(xx_test, yy_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (128, 128, 1)

In [1]:
par1={
    'Validation': 0.2, 
    'epoch': 40, 
    'Batch': 10
}
par = {
    'filter1':64,
    'filter2':128,
    'strides':1,
    'kernel': 2,
    'dropout1':0.5,
    'dropout2':0.5,
    'Neural':256
}

def DL_fun1(par,par1,x_train,y_train,x_test,y_test):
    result = None
    nf=par['filter1']
    nf1=par['filter2']
    ks=par['kernel']
    hs=np.shape(x)[1]
    ws=np.shape(x)[2]
    dr1=par['dropout1']
    dr2=par['dropout2']
    nn=par['Neural']
    vsr=par1['Validation']
    epn=par1['epoch']
    bs=par1['Batch']
    y_train_onehot = to_categorical(y_train)
    y_test_onehot = to_categorical(y_test)
    try:
        # 只使用 80% 的 GPU 記憶體config = tf.ConfigProto()
#         gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5, allow_growth=True)
#         sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
#         tf.keras.backend.set_session(sess)

        model = Sequential([
            Conv2D(filters=nf,kernel_size=(ks,ks),strides=(par['strides'],par['strides']),padding='same',input_shape=(hs,ws,1),activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(filters=nf1,kernel_size=(ks,ks),strides=(par['strides'],par['strides']),padding='same',input_shape=(hs,ws,1),activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Dropout(dr1),
            Flatten(),
            Dense(nn, activation='relu'),
            Dropout(dr2),
            Dense(3,activation='softmax'),
        ])
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        print(model.summary()) 
        train_history=model.fit(x=x_train,
                                y=y_train_onehot,
                                validation_split=vsr,
                                epochs=epn,
                                batch_size=bs,
                                verbose=2)
        scores=model.evaluate(x=x_test,y=y_test_onehot)
        result={'History':train_history, 'Score': scores}
        print(result)
    except Exception as e:
        print(traceback.format_exc())
    finally:
#         sess.close()
        pass
    return result

In [ ]:
result=DL_fun1(par,par1,x_train,y_train,x_test,y_test)

In [41]:
print(result)

None
